In [38]:
import os
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
from bs4 import BeautifulSoup
import requests

In [39]:
round_ = '17'
url = f'https://www.sportsbet.com.au/betting/australian-rules/afl/round-{round_}'
url

'https://www.sportsbet.com.au/betting/australian-rules/afl/round-17'

In [40]:
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

In [41]:
tmp = soup.find_all('script')

In [46]:
game_urls = []
for i in tmp[:9]:
    try:
        url_find = re.compile(r'https://www.sportsbet.com.au/betting/australian-rules/afl/(.*?)\-\d{7}')
        game_url = url_find.search(i.text)
        game_urls.append(game_url.group())
    except:
        continue
game_urls

['https://www.sportsbet.com.au/betting/australian-rules/afl/western-bulldogs-v-collingwood-7454970',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/brisbane-v-west-coast-7454971',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/greater-western-sydney-v-hawthorn-7454980',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/st-kilda-v-melbourne-7454981',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/port-adelaide-v-gold-coast-7455001',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/geelong-v-north-melbourne-7455010',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/essendon-v-adelaide-7455011',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/fremantle-v-carlton-7455012']

In [47]:
r = re.compile(r'\/(\w*\-?\w*\-?\w*)\-v\-(\w*\-?\w*\-?\w*)\-')
game_list = []
for game in game_urls:
    game_list.append(r.findall(game))

In [48]:
game_list

[[('western-bulldogs', 'collingwood')],
 [('brisbane', 'west-coast')],
 [('greater-western-sydney', 'hawthorn')],
 [('st-kilda', 'melbourne')],
 [('port-adelaide', 'gold-coast')],
 [('geelong', 'north-melbourne')],
 [('essendon', 'adelaide')],
 [('fremantle', 'carlton')]]

In [49]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.set_window_size(700,800)

[WDM] - Downloading: 100%|█████████████████| 8.29M/8.29M [00:00<00:00, 9.57MB/s]


In [50]:
disposals = ['To Get 15 or More Disposals',
            'To Get 20 or More Disposals',
            'To Get 25 or More Disposals',
            'To Get 30 or More Disposals',]
output = pd.DataFrame({},columns = {'teams','round','Player', 'disposals', 'odds'})

In [51]:
for game, game_url in zip(game_list, game_urls):
    driver.get(game_url)
    try:
        first_click = driver.find_element_by_xpath('(//*[@class="size14_f7opyze medium_f1wf24vo"])')
        first_click.click()
        
        for i in range(1,7):
            if driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze medium_f1wf24vo"])[{i}]').text == 'Disposal Markets':
                disposal_market = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze medium_f1wf24vo"])[{i}]')
        
        
        disposal_market.click()
        time.sleep(3)
        
        for i in range(5,50):
            try:
                tmp = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze bold_f1au7gae"])[{i}]').text
            except:
                continue
            if tmp in disposals:

                dis = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze bold_f1au7gae"])[{i}]')

                dis.click()

                player = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg outcomeDetailsFirst_f162fqwy"])').text.split('\n')
                print(player)
                new_row = {'Player': player[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':tmp,'odds':player[-1]}
                output.loc[len(output)] = new_row

                for j in range(1,50):
                    try:

                        tmp1 = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg"])[{j}]').text.split('\n')
                        print(tmp1)
                        new_row = {'Player': tmp1[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':tmp,'odds':tmp1[-1]}
                        output.loc[len(output)] = new_row
                    except:
                        continue
                
                dis.click()
                time.sleep(2)
            
    except:
        continue

['Caleb Daniel', 'SUS']
['Ed Richards', '1.15']
['Tim English', '1.15']
['Bailey Williams (WB)', '1.44']
['Caleb Poulter', '1.95']
['Laitham Vandermeer', '2.30']
['Jamarra Ugle-Hagan', '2.50']
['Taylor Duryea', '2.90']
['Alex Keath', '2.95']
['Aaron Naughton', '3.30']
['Cody Weightman', '3.90']
['Jordan De Goey', '1.08']
['John Noble', '1.10']
['Patrick Lipinski', '1.15']
['Taylor Adams', '1.17']
['Jack Crisp', '1.19']
['Isaac Quaynor', '1.36']
['Jeremy Howe', '1.39']
['Darcy Moore', '1.44']
['Darcy Cameron', '1.58']
['Will Hoskin-Elliott', '1.78']
['Oleg Markov', '2.12']
['Jamie Elliott', '3.90']
['Tom Liberatore', '1.09']
['Jack Macrae', '1.10']
['Marcus Bontempelli', '1.10']
['Bailey Smith', '1.20']
['Bailey Dale', '1.22']
['Caleb Daniel', '1.29']
['Tim English', '1.82']
['Ed Richards', '1.94']
['Bailey Williams (WB)', '2.60']
['Caleb Poulter', '5.90']
['Laitham Vandermeer', '6.25']
['Jamarra Ugle-Hagan', '9.00']
['Alex Keath', '9.75']
['Tom Mitchell', '1.09']
['Josh Daicos', '1.16'

['Ed Langdon', '1.07']
['James Jordon', '1.11']
['Trent Rivers', '1.12']
['Steven May', '1.13']
['Tom Sparrow', '1.18']
['Jake Bowey', '1.22']
['Taj Woewodin', '1.35']
['Max Gawn', '1.44']
['Jake Lever', '1.58']
['Alex Neal-Bullen', '1.64']
['Brodie Grundy', '1.67']
['Judd McVee', '1.68']
['Charlie Spargo', '2.35']
['Kysaiah Pickett', '2.60']
['Jake Melksham', '4.60']
['Brad Crouch', '1.10']
['Jack Steele', '1.36']
['Nasiah Wanganeen-Milera', '1.36']
['Mason Wood', '1.70']
['Jack Billings', '1.95']
['Zak Jones', '1.96']
['Rowan Marshall', '2.08']
['Sebastian Ross', '2.25']
['Marcus Windhager', '2.50']
['Callum Wilkie', '2.60']
['Liam Stocker', '3.30']
['Jade Gresham', '3.60']
['Jimmy Webster', '4.00']
['Mitchito Owens', '4.75']
['Dougal Howard', '6.50']
['Mattaes Phillipou', '9.75']
['Angus Brayshaw', '1.15']
['Jack Viney', '1.16']
['Lachie Hunter', '1.22']
['Christian Salem', '1.27']
['Ed Langdon', '1.49']
['James Jordon', '1.68']
['Trent Rivers', '1.72']
['Steven May', '1.94']
['Tom 

['Chayce Jones', '1.34']
['Lachlan Sholl', '1.42']
['Josh Rachele', '1.43']
['Izak Rankine', '1.44']
['Jake Soligo', '1.55']
['Lachlan Murphy', '1.55']
['Ned McHenry', '1.57']
['Josh Worrell', '1.59']
["Reilly O'Brien", '1.64']
['Taylor Walker', '2.10']
['Riley Thilthorpe', '2.30']
['Luke Pedlar', '2.80']
['Zach Merrett', 'SUS']
['Dylan Shiel', '1.28']
['Nic Martin', '1.39']
['Ben Hobbs', '1.40']
['Jye Caldwell', '1.43']
['Mason Redman', '1.55']
['Dyson Heppell', '1.68']
['Andrew McGrath', '1.71']
['Jordan Ridley', '1.72']
['Jake Stringer', '3.05']
['Archie Perkins', '4.30']
['Sam Durham', '4.30']
['Jayden Laverde', '4.50']
['Kyle Langford', '4.80']
['Jake Kelly', '5.00']
['Will Snelling', '10.50']
['Ben Keays', '1.08']
['Mitch Hinge', '1.70']
['Brodie Smith', '1.78']
['Rory Sloane', '1.86']
['Wayne Milera', '2.10']
['Chayce Jones', '2.20']
['Izak Rankine', '2.55']
['Josh Rachele', '2.95']
['Lachlan Sholl', '3.00']
['Ned McHenry', '3.10']
['Jake Soligo', '3.60']
['Josh Worrell', '3.90'

In [27]:
output

,disposals,teams,odds,Player,round
0,To Get 15 or More Disposals,richmond v sydney,SUS,dion prestia,R17
1,To Get 15 or More Disposals,richmond v sydney,SUS,daniel rioli,R17
2,To Get 15 or More Disposals,richmond v sydney,1.10,nick vlastuin,R17
3,To Get 15 or More Disposals,richmond v sydney,1.13,shai bolton,R17
4,To Get 15 or More Disposals,richmond v sydney,1.15,dustin martin,R17
...,...,...,...,...,...
818,To Get 30 or More Disposals,fremantle v carlton,6.00,blake acres,R17
819,To Get 30 or More Disposals,fremantle v carlton,8.50,matthew kennedy,R17
820,To Get 30 or More Disposals,fremantle v carlton,10.50,adam saad,R17
821,To Get 30 or More Disposals,fremantle v carlton,11.00,nic newman,R17


In [28]:
output['home'] = output['teams'].apply(lambda x: x.split(' v ')[0])
output['away'] = output['teams'].apply(lambda x: x.split(' v ')[1])

In [34]:
team_mapping = {'richmond':'RCH','geelong':'GEE',
                'north-melbourne':'NTH','port-adelaide':'POR',
                'gold-coast':'GCS','melbourne':'MEL',
                'greater-western-sydney':'GWS','hawthorn':'HAW',
                'carlton':'CAR','brisbane':'BRL',
                'collingwood':'COL','adelaide':'ADE',
                'fremantle':'FRE','western-bulldogs':'WBD',
                'sydney':'SYD','west-coast':'WCE',
                'st-kilda':'STK','essendon':'ESS'}

In [35]:
for key in team_mapping.keys():
     output.loc[output['home']==key, 'home'] = team_mapping[key]
for key in team_mapping.keys():
    output.loc[output['away']==key, 'away'] = team_mapping[key]     

In [36]:
output

,disposals,teams,odds,Player,round,home,away
0,To Get 15 or More Disposals,richmond v sydney,SUS,dion prestia,R17,RCH,SYD
1,To Get 15 or More Disposals,richmond v sydney,SUS,daniel rioli,R17,RCH,SYD
2,To Get 15 or More Disposals,richmond v sydney,1.10,nick vlastuin,R17,RCH,SYD
3,To Get 15 or More Disposals,richmond v sydney,1.13,shai bolton,R17,RCH,SYD
4,To Get 15 or More Disposals,richmond v sydney,1.15,dustin martin,R17,RCH,SYD
...,...,...,...,...,...,...,...
818,To Get 30 or More Disposals,fremantle v carlton,6.00,blake acres,R17,FRE,CAR
819,To Get 30 or More Disposals,fremantle v carlton,8.50,matthew kennedy,R17,FRE,CAR
820,To Get 30 or More Disposals,fremantle v carlton,10.50,adam saad,R17,FRE,CAR
821,To Get 30 or More Disposals,fremantle v carlton,11.00,nic newman,R17,FRE,CAR


In [37]:
output.to_csv('/Users/caseyh/Desktop/footyfootyfooty/sportsbet_odds/test_disposal_markets_r17.csv', index = False)